# 自然语言识别处理单元
## [拂晓工作室](https://github.com/Errrneist/Alchemist)
* 此程序可用于识别文字中的语言以及深度分析新闻概况

# 参考资料
* [1] [SFrame入门](https://apple.github.io/turicreate/docs/api/generated/turicreate.SFrame.html#turicreate.SFrame)
* [2] [从字符串中删除子字符串](https://stackoverflow.com/questions/31273642/better-way-to-remove-multiple-words-from-a-string)
* [3] [用Python分析《红楼梦》 - 楼宇](https://zhuanlan.zhihu.com/p/29209681)
* [4] [使用pyltp包进行中文分词实例](https://blog.csdn.net/sinat_26917383/article/details/77067515)
* [5] [pyltp技术文档](http://pyltp.readthedocs.io/zh_CN/latest/api.html#id15)
* [6] [pyltp介绍文档一](https://www.ltp-cloud.com/intro/#introduction )
* [7] [pyltp介绍文档二](http://ltp.readthedocs.io/zh_CN/latest/appendix.html#id5)
* [8] [pyltp深度训练模型](https://pan.baidu.com/share/link?shareid=1988562907&uk=2738088569#list/path=/)


# 导入库

In [1]:
# 导入库
import urllib
import re
import pymysql
import os
import csv
import time
import datetime
# import pyltp
import turicreate as tc
from bs4 import BeautifulSoup

/Users/hongjunwu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
news = tc.SFrame('../DataSets/Eastmoney/China/20180630-2015.csv')

Finished parsing file /Users/hongjunwu/Documents/GitHub/Alchemist/DataSets/Eastmoney/China/20180630-2015.csv

Parsing completed. Parsed 100 lines in 0.064344 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,int,list,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/hongjunwu/Documents/GitHub/Alchemist/DataSets/Eastmoney/China/20180630-2015.csv

Parsing completed. Parsed 515 lines in 0.036906 secs.

In [5]:
news

contents,day,month,related
中国人大网6月29日透露，第十三届全国人大常委会第三次会议日前已对《中华人民共和国个人所得税 ...,29,6,[]
中国经济导报中国发展网粤工商马思泳记者皮泽红报道近日，广东省工商局等16部门联合发布了《广东 ...,29,6,[房地产]
【22项重大举措 新外资准入负面清单释放开放新信号】2018年版外商投资准入负面清单，在一、 ...,29,6,[]
国家市场监管总局昨日通报称，北京市交管局在交通违章罚款管理中未经公开竞争性程序指定工商银行作 ...,28,6,"[工商银行, 银行, 工商银行]"
内蒙古自治区党委书记、自治区协调联络工作领导小组组长李纪恒主持召开中央环境保护督察“回头看” ...,28,6,[]
【商务部：中欧经贸合作将为全球经济复苏带来“暖流”】商务部新闻发言人高峰28日说，中欧双方在 ...,28,6,[]
城市副中心与廊坊北三县地区如何协同发展？城市副中心如何辐射带动北三县？京津冀区域协同发展示范 ...,27,6,[房地产]
【石家庄：允许高库存地区商服用地改性为住宅 比例不超过七成】6月29日，河北省石家庄市人民政 ...,29,6,[房地产]
据新华社报道，针对近期房地产市场乱象，为严厉打击侵害群众利益的违法违规行为，住房和城乡建设部 ...,28,6,[房地产]
正式获批为国家中心城市，可以称得上是西安今年最为重大的利好之一。在国家发改委和住建部正式批复 ...,29,6,[长和]


In [3]:
news[2]

{'contents': '【22项重大举措 新外资准入负面清单释放开放新信号】2018年版外商投资准入负面清单，在一、二、三产业全面放宽市场准入，涉及金融、交通运输、商贸流通、专业服务、制造、基础设施、能源、资源、农业等各领域，共推出22项重大开放措施。（新华社）国家发展改革委、商务部28日对外发布《外商投资准入特别管理措施(负面清单)(2018年版)》，大幅放开22个领域外商投资市场准入。新版外商投资准入负面清单有哪些重大开放举措？以更大力度推动新一轮对外开放意义何在？记者采访了相关人士。22项重大举措推动全方位开放2018年版外商投资准入负面清单，在一、二、三产业全面放宽市场准入，涉及金融、交通运输、商贸流通、专业服务、制造、基础设施、能源、资源、农业等各领域，共推出22项重大开放措施。“修订负面清单，总的原则是以更大力度推进对外开放，不仅是清单长度要缩短，更重要的是推动重点领域开放，推出有标志意义的举措。”国家发展改革委有关负责人说。党的十八大以来，我国逐步形成和完善了自贸试验区的外商投资准入负面清单，并在2017年首次提出在全国范围内实施的外商投资准入负面清单。近年来，外资限制措施已从180项左右减少至60多项，减少近三分之二。2018年版负面清单比2017年版的63条减少了15条，进一步缩小了外商投资审批范围。“对外商投资实行准入前国民待遇加负面清单管理模式，是中国适应经济全球化新形势和国际投资规则变化的制度变革。”国务院新闻办公室28日发表的《中国与世界贸易组织》白皮书这样评价。愈加广阔开放的市场、不断优化的营商环境让中国成为富有吸引力的投资热土。今年1至5月，发达经济体对华投资增长显著。新加坡、韩国、日本、美国、英国等对华实际投入金额同比分别增长33.9%、66.9%、5.1%、16.3%和56.9%。中国美国商会《2018中国商务环境调查报告》显示，约60%受访企业将中国列为全球三大投资目的地之一，74%的会员企业计划于2018年扩大在华投资，比例为近年来最高。中国宏观经济研究院对外经济研究所副研究员李大伟说，此次修订负面清单，再次大幅放宽外资市场准入，不仅取消了汽车、飞机、船舶等制造业领域外资准入限制，还在金融、运输等服务业及种业等领域进一步开放，充分体现了我国外资管理体制更加开放、灵活、高效的改革方向，对构建开放型经济新体制具有重大意义。对汽车